<a href="https://colab.research.google.com/github/wenchun0731/github-colab/blob/main/%E6%B8%AC%E8%A9%A6%E9%A1%AF%E7%A4%BA%E5%BA%A7%E6%A8%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 在Google Colab上运行此代码前，请确保您已经上传了您的视频文件。

# 安装所需库
!pip install --upgrade pip
!pip install opencv-python opencv-python-headless torch torchvision numpy==1.23.5

# 克隆YOLOv7仓库
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7
!pip install -r requirements.txt

import torch
import cv2
import numpy as np
import pandas as pd
import os
from google.colab.patches import cv2_imshow

# 下载YOLOv7权重文件
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

# 加载YOLOv7模型
model = torch.hub.load('WongKinYiu/yolov7', 'custom', 'yolov7.pt')
model.eval()

# 定义追踪函数
def detect_objects(frame):
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()
    boxes = []
    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection
        if confidence > 0.5 and class_id == 2:  # 2 是 COCO 数据集中 'car' 类别的 ID
            boxes.append([int(x1), int(y1), int(x2 - x1), int(y2 - y1)])
    return boxes

# 打开视频
video_path = '/content/drive/MyDrive/test-video-log2.mp4'  # 替换为您的视频路径
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video opened successfully.")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Frame Width: {frame_width}, Frame Height: {frame_height}, FPS: {fps}")

# 设置输出视频的参数
output_path = '/content/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print("Error: Could not open video writer.")
else:
    print("Video writer opened successfully.")

coordinates = []

frame_count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  # 当前帧的索引
    boxes = detect_objects(frame)
    for box in boxes:
        x, y, w, h = box
        coordinates.append({
            'frame_id': frame_id,
            'x': x,
            'y': y,
            'w': w,
            'h': h
        })
        # 在每个框上绘制矩形和坐标
        color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f'car: {x}, {y}, {w}, {h}', (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, color, 2)
    out.write(frame)  # 将处理后的帧写入输出视频
    frame_count += 1
    if frame_count % 10 == 0:
        print(f"Processed {frame_count} frames.")
        cv2_imshow(frame)  # 在Colab上显示图像

cap.release()
out.release()

# 检查文件是否已创建
if os.path.exists(output_path):
    print(f"Output video saved successfully at {output_path}")
else:
    print(f"Error: Output video was not saved at {output_path}")

# 保存坐标数据
csv_path = '/content/vehicle_coordinates.csv'
df = pd.DataFrame(coordinates)
df.to_csv(csv_path, index=False)
print(f"Coordinates have been saved to {csv_path}")

# 下载生成的视频文件
from google.colab import files
files.download(output_path)
